In [12]:
import scanpy as sc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import openpyxl

In [13]:
adata = sc.read_h5ad("/Users/cyluo/Library/CloudStorage/Box-Box/MERFISH Collaboration 3/clustering3/h5ad files/zenodo/merscope_integrated_855.h5ad")
print(adata)

AnnData object with n_obs × n_vars = 15927370 × 300
    obs: 'gw', 'sample', 'region', 'area', 'layer', 'H1_annotation', 'H2_annotation', 'H3_annotation', 'relative_height', 'cortical_depth'
    uns: 'rank_genes_groups'
    obsm: 'spatial'


/Users/cyluo/anaconda3/envs/xuyuEnv/lib/python3.9/site-packages/anndata/_core/anndata.py:1897: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [14]:
adata.obs.index = adata.obs.index.map(lambda x: x.split('-')[0])
print(adata)

AnnData object with n_obs × n_vars = 15927370 × 300
    obs: 'gw', 'sample', 'region', 'area', 'layer', 'H1_annotation', 'H2_annotation', 'H3_annotation', 'relative_height', 'cortical_depth'
    uns: 'rank_genes_groups'
    obsm: 'spatial'


In [17]:
# 1. Creating the 'sample_region' column
adata.obs['sample_region'] = adata.obs['sample'].astype(str) + '-' + adata.obs['region'].astype(str)
print(adata.obs)

                     gw   sample region area layer H1_annotation  \
2241155200012100016  20    FB080     F1  NaN   NaN         EN-IT   
2241155200012100018  20    FB080     F1  NaN   NaN         EN-IT   
2241155200012100021  20    FB080     F1  NaN   NaN        EN-Mig   
2241155200012100028  20    FB080     F1  NaN   NaN         EN-IT   
2241155200012100030  20    FB080     F1  NaN   NaN         EN-IT   
...                  ..      ...    ...  ...   ...           ...   
2655870701320100026  34  UMB5900  BA40b  NaN   NaN         EN-ET   
2655870701326100057  34  UMB5900  BA40b  NaN   NaN        EN-Mig   
2655870701326100064  34  UMB5900  BA40b  NaN   NaN          Glia   
2655870701326100070  34  UMB5900  BA40b  NaN   NaN            EC   
2655870701326100096  34  UMB5900  BA40b  NaN   NaN          Glia   

                    H2_annotation  H3_annotation  relative_height  \
2241155200012100016    EN-IT-L2/3    EN-IT-L3-V1              NaN   
2241155200012100018    EN-IT-L2/3    EN-IT-L3

In [18]:
plot_df = adata.obs
merged_encluster_sr_H2 = plot_df.groupby(['sample_region', 'H2_annotation']).size().reset_index(name='sr_count')
merged_encluster_sr_H2.to_csv("encluster_sr_H2.csv", index=False)

/var/folders/xz/tff3yvr92zz54zqk2m9jp1f00000gq/T/ipykernel_28028/1894234099.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged_encluster_sr_H2 = plot_df.groupby(['sample_region', 'H2_annotation']).size().reset_index(name='sr_count')


In [19]:
merged_encluster_H2 = plot_df.groupby(['H2_annotation']).size().reset_index(name='H2_count')
merged_encluster_H2.to_csv("encluster_H2.csv", index=False)

/var/folders/xz/tff3yvr92zz54zqk2m9jp1f00000gq/T/ipykernel_28028/2322436814.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged_encluster_H2 = plot_df.groupby(['H2_annotation']).size().reset_index(name='H2_count')


In [20]:
count_sr_H2 = pd.merge(merged_encluster_sr_H2, merged_encluster_H2, on="H2_annotation")
count_sr_H2['percent'] = count_sr_H2['sr_count'] / count_sr_H2['H2_count']
print(count_sr_H2.head)

<bound method NDFrame.head of       sample_region H2_annotation  sr_count  H2_count   percent
0          FB080-F1       Astro-1      7620    291042  0.026182
1         FB080-F2a       Astro-1      8698    291042  0.029886
2         FB080-F2b       Astro-1      6803    291042  0.023375
3         FB080-O1a       Astro-1      1413    291042  0.004855
4         FB080-O1b       Astro-1      1306    291042  0.004487
...             ...           ...       ...       ...       ...
1282   UMB5900-BA22      vRG-late     13338    278120  0.047958
1283    UMB5900-BA4      vRG-late      8397    278120  0.030192
1284  UMB5900-BA40a      vRG-late     11777    278120  0.042345
1285  UMB5900-BA40b      vRG-late     10586    278120  0.038063
1286    UMB5900-BA9      vRG-late     10563    278120  0.037980

[1287 rows x 5 columns]>


In [21]:
# Check the sum of the 'percent' column for each 'H2_annotation'
percent_sum_check = count_sr_H2.groupby('H2_annotation')['percent'].sum().reset_index(name='percent_sum')

# Display the sums to check if any do not equal 1
print(percent_sum_check)

     H2_annotation  percent_sum
0          Astro-1          1.0
1      Astro-late1          1.0
2               EC          1.0
3       EN-ET-L5/6          1.0
4   EN-ET-L6-early          1.0
5         EN-ET-SP          1.0
6       EN-ET-SP-P          1.0
7   EN-ET-SP-early          1.0
8       EN-IT-L2/3          1.0
9       EN-IT-L3/4          1.0
10        EN-IT-L4          1.0
11      EN-IT-L4/5          1.0
12        EN-IT-L6          1.0
13         EN-IZ-1          1.0
14         EN-IZ-2          1.0
15           EN-L2          1.0
16       EN-oSVZ-1          1.0
17       En-oSVZ-2          1.0
18          IN-CGE          1.0
19          IN-MGE          1.0
20          IN-SST          1.0
21        IN-VZ/GE          1.0
22       INP-VZ/GE          1.0
23       IPC-SVZ-1          1.0
24       IPC-SVZ-2          1.0
25      IPC-VZ/SVZ          1.0
26        IPC-iSVZ          1.0
27        IPC-oSVZ          1.0
28             OPC          1.0
29             RG1          1.0
30      

/var/folders/xz/tff3yvr92zz54zqk2m9jp1f00000gq/T/ipykernel_28028/2531004775.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  percent_sum_check = count_sr_H2.groupby('H2_annotation')['percent'].sum().reset_index(name='percent_sum')


In [22]:
count_sr_H2.to_csv("H2sr_histogram.csv")